In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

train_raw = pd.read_csv("train.csv")
test_raw = pd.read_csv("test.csv")
print('raw train data set size ')
print(train_raw.shape)
print('raw test data set size ')
print(test_raw.shape)
 
X = pd.concat((train_raw.loc[:,'MSSubClass':'SaleCondition'], test_raw.loc[:,'MSSubClass':'SaleCondition']))

Y = train_raw.SalePrice

#Dropping irrelelvant attributes where almost 1300 rows are Null values
X.drop(['GarageYrBlt','Alley','PoolQC','Fence','MiscFeature','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','BsmtFinSF2','LowQualFinSF','BsmtHalfBath'],axis=1,inplace=True)
# fill the missing value automatically


#Filling in with mean
X['LotFrontage']=X['LotFrontage'].fillna(X['LotFrontage'].mean())

#Filling in with mode
X['BsmtCond']=X['BsmtCond'].fillna(X['BsmtCond'].mode()[0])
X['BsmtQual']=X['BsmtQual'].fillna(X['BsmtQual'].mode()[0])
X['FireplaceQu']=X['FireplaceQu'].fillna(X['FireplaceQu'].mode()[0])
X['GarageType']=X['GarageType'].fillna(X['GarageType'].mode()[0])
X['MasVnrType']=X['MasVnrType'].fillna(X['MasVnrType'].mode()[0])
X['MasVnrArea']=X['MasVnrArea'].fillna(X['MasVnrArea'].mode()[0])

X = X.fillna(X.mean()) 

# separet data back to remvoe outiers from training set
train = X[:len(train_raw)]
X_test = X[train_raw.shape[0]:]
Y_train = Y
print("before")
print(X_test.shape)

# find the nomirecal attribute to identify outliers (creat df with number only)

num_train = train.select_dtypes(include=["number"])
cat_train = train.select_dtypes(exclude=["number"])

#find Q1, Q3, and interquartile range for each column
Q1 = num_train.quantile(0.25)
Q3 = num_train.quantile(0.75)
IQR = Q3 - Q1
idx = ~((num_train < (Q1 - 1.5 * IQR)) | (num_train > (Q3 + 1.5 * IQR))).any(axis=1)
train_cleaned = pd.concat([num_train.loc[idx], cat_train.loc[idx]], axis=1)


print('training set size after cleaning ')
print(train_cleaned.shape)

# convert categorical/discreate variables 
df = pd.concat([train_cleaned,X_test])
df = pd.get_dummies(df)

X_train = df[:len(train_cleaned)]
X_test = df[train_cleaned.shape[0]:]
Y_train = Y_train[idx]


print('Xtrain shape')
print(X_train.shape)

print('Xttest shape')
print(X_test.shape)

print('Yrain shape')
print(Y_train.shape)
print(df.shape)
#rint(train)
print("after")
print(X_test.shape)

raw train data set size 
(1460, 81)
raw test data set size 
(1459, 80)
before
(1459, 66)
training set size after cleaning 
(900, 66)
Xtrain shape
(900, 256)
Xttest shape
(1459, 256)
Yrain shape
(900,)
(2359, 256)
after
(1459, 256)


/var/folders/0q/14n26x893mj9929smn52_s840000gn/T/ipykernel_80324/3847779197.py:33: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X = X.fillna(X.mean())


In [2]:
from sklearn.linear_model import Lasso

reg = Lasso(alpha=0.1)
reg.fit(X_train, Y_train)

name = "lasso"
train_score = reg.score(X_train, Y_train)
#val_score = reg.score(X_val, Y_val)

print(name, "training score (MSE)=", train_score)

print()

lasso training score (MSE)= 0.9446814539764274



/Users/abdulfattah/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.282e+10, tolerance: 2.995e+08
  model = cd_fast.enet_coordinate_descent(


In [3]:
 print(format('How we can find parameters using GridSearchCV  for Regression','*^82'))

    # loading libraries
    #from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor


    #Using GridSearchCV
GBR = GradientBoostingRegressor()
parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
            'subsample'    : [0.9, 0.5, 0.2, 0.1],
            'n_estimators' : [100,500,1000, 1500],
            'max_depth'    : [4,6,8,10]
             }
grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 5, n_jobs=-1)
grid_GBR.fit(X_train, Y_train)

    # Results from Grid Search
print(" Results from Grid Search " )
print()
print("\n The best estimator across ALL searched params:\n", grid_GBR.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_GBR.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_GBR.best_params_)

**********How we can find parameters using GridSearchCV  for Regression***********
 Results from Grid Search 


 The best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.01, max_depth=8, n_estimators=1500,
                          subsample=0.2)

 The best score across ALL searched params:
 0.9139561076346692

 The best parameters across ALL searched params:
 {'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 1500, 'subsample': 0.2}


In [4]:
# import dependices 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
# split data
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train)

# creat
GBR = GradientBoostingRegressor(learning_rate = 0.03, max_depth = 8, n_estimators = 500, subsample = 0.2)
# fit 
GBR.fit(X_train, Y_train)
Y_predict = GBR.predict(X_val)
Y_predict[Y_predict <= 0] = 1

# check preformance 
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(np.log(Y_val), np.log(Y_predict))
rmse = np.sqrt(mse)

print(rmse)

#predict
Y_pred = GBR.predict(X_test)
Y_pred[Y_pred <= 0] = 1
pred_df = pd.DataFrame(Y_pred, index=test_raw["Id"], columns=["SalePrice"])
pred_df.to_csv('output_GBRGRid.csv', header=True, index_label='Id')

0.11518726375649455


In [5]:
print(format('How we can find parameters using GridSearchCV  for Regression','*^82'))

    # loading libraries
    #from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor

    # loading the diabetes datasets
    #dataset = datasets.load_diabetes()
    #X = dataset.data; y = dataset.target
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

    #Using GridSearchCV
GBR = GradientBoostingRegressor()
parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
            'subsample'    : [0.9, 0.5, 0.2, 0.1],
            'n_estimators' : [100,500,1000, 1500],
            'max_depth'    : [4,6,8,10]
             }
grid_GBR = RandomizedSearchCV(GBR, parameters, cv = 5, n_jobs=-1)
grid_GBR.fit(X_train, Y_train)

    # Results from Grid Search
print(" Results from Grid Search " )
print()
print("\n The best estimator across ALL searched params:\n", grid_GBR.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_GBR.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_GBR.best_params_)

**********How we can find parameters using GridSearchCV  for Regression***********
 Results from Grid Search 


 The best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.04, max_depth=4, n_estimators=500,
                          subsample=0.2)

 The best score across ALL searched params:
 0.9041862151162734

 The best parameters across ALL searched params:
 {'subsample': 0.2, 'n_estimators': 500, 'max_depth': 4, 'learning_rate': 0.04}


In [6]:
# import dependices 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
# split data
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train)

# creat
GBR = GradientBoostingRegressor(learning_rate = 0.01, max_depth = 4, n_estimators = 1500, subsample = 0.5)
# fit 
GBR.fit(X_train, Y_train)
Y_predict = GBR.predict(X_val)
Y_predict[Y_predict <= 0] = 1

# check preformance 
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(np.log(Y_val), np.log(Y_predict))
rmse = np.sqrt(mse)

print(rmse)

#predict
Y_pred = GBR.predict(X_test)
Y_pred[Y_pred <= 0] = 1
pred_df = pd.DataFrame(Y_pred, index=test_raw["Id"], columns=["SalePrice"])
pred_df.to_csv('output_GBR.csv', header=True, index_label='Id')

0.09107069583953989
